In [42]:
import pandas as pd
df = pd.read_csv('C:/Users/CHK/Desktop/Final_Project/price_prediction/model7/processed_ko_4.csv')
df.drop(['Unnamed: 0', 'SEQ', 'LMNAME', 'VNUM', 'MNAME'], axis=1, inplace=True)

111

In [120]:
# 색상 컬럼의 각 셀에 대해 조건부로 값을 대체하는 함수 정의
def replace_color(text):
    if '청색' in text:
        return '파란색'
    else:
        return text
# 색상 컬럼의 각 셀에 대해 조건부로 값을 대체하여 새로운 컬럼에 저장
df['COLOR'] = df['COLOR'].apply(replace_color)

unique_values = df['COLOR'].unique().tolist()
print(len(unique_values))
print(unique_values)

20
['흰색', '검정색', '은색', '회색', '파란색', '기타색상', '초록색', '빨간색', '남색', '갈색', '자주색', '하늘색', '노란색', '연두색', '금색', '주황색', '보라색', '무광', '분홍색', '밤색']


In [122]:
df.to_csv('color_processed.csv')

In [126]:
df

,PRICE,MYERAR,MILEAGE,COLOR,TRANS,F_TYPE,DISP,VTYPE,CU_HIS,MVD_HIS,AVD_HIS,TL_HIS,FD_HIS,VT_HIS,US_HIS
0,173.0,2018,80800.0,흰색,자동,디젤,2497.0,트럭,NaN,1.0,5.0,0.0,0.0,0.0,4.0
1,522.0,2022,52600.0,검정색,자동,가솔린,3342.0,대형,NaN,0.0,0.0,0.0,0.0,0.0,1.0
2,95.0,2012,156291.0,흰색,자동,가솔린,1998.0,스포츠카,NaN,1.0,1.0,0.0,0.0,0.0,7.0
3,63.0,2010,174361.0,검정색,자동,가솔린,1998.0,스포츠카,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57.9,2016,122000.0,흰색,자동,디젤,1597.0,SUV,NaN,5.0,3.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043,2390.0,2021,72162.0,기타색상,오토,디젤,0.0,SUV,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6044,1190.0,2018,82286.0,검정색,오토,가솔린,0.0,준중형,1.0,0.0,0.0,0.0,0.0,0.0,4.0
6045,1390.0,2018,131512.0,흰색,오토,하이브리드(가솔린),1490.0,소형,0.0,0.0,0.0,0.0,0.0,0.0,3.0
6046,560.0,2010,113517.0,갈색,오토,가솔린,1591.0,준중형,0.0,0.0,0.0,0.0,0.0,0.0,8.0


In [129]:
selected_columns = ['COLOR', 'TRANS', 'F_TYPE', 'VTYPE']

# 선택한 컬럼들로부터 더미 변수 생성
dummy_df = pd.get_dummies(df[selected_columns])

# 선택한 컬럼들을 제외한 나머지 컬럼들로 데이터프레임 생성
other_columns = [col for col in df.columns if col not in selected_columns]
other_df = df[other_columns]

# 선택한 컬럼들과 나머지 컬럼들을 결합
final_df = pd.concat([other_df, dummy_df], axis=1)

print(final_df)

       PRICE  MYERAR   MILEAGE    DISP  CU_HIS  MVD_HIS  AVD_HIS  TL_HIS  \
0      173.0    2018   80800.0  2497.0     NaN      1.0      5.0     0.0   
1      522.0    2022   52600.0  3342.0     NaN      0.0      0.0     0.0   
2       95.0    2012  156291.0  1998.0     NaN      1.0      1.0     0.0   
3       63.0    2010  174361.0  1998.0     NaN      NaN      NaN     NaN   
4       57.9    2016  122000.0  1597.0     NaN      5.0      3.0     0.0   
...      ...     ...       ...     ...     ...      ...      ...     ...   
6043  2390.0    2021   72162.0     0.0     1.0      0.0      0.0     0.0   
6044  1190.0    2018   82286.0     0.0     1.0      0.0      0.0     0.0   
6045  1390.0    2018  131512.0  1490.0     0.0      0.0      0.0     0.0   
6046   560.0    2010  113517.0  1591.0     0.0      0.0      0.0     0.0   
6047  1850.0    2020   31502.0     0.0     0.0      0.0      0.0     0.0   

      FD_HIS  VT_HIS  ...  VTYPE_대형  VTYPE_버스  VTYPE_소형  VTYPE_스포츠카  VTYPE_승  \
0      

In [133]:
final_df = final_df.fillna(0)

In [135]:
final_df.dtypes

PRICE        float64
MYERAR         int64
MILEAGE      float64
DISP         float64
CU_HIS       float64
              ...   
VTYPE_승합        bool
VTYPE_준중형       bool
VTYPE_중형        bool
VTYPE_트럭        bool
VTYPE_특수        bool
Length: 64, dtype: object

In [159]:
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# 원본 데이터프레임 복사
df_copy = final_df.copy()

# 1. K-means 클러스터링
kmeans = KMeans(n_clusters=7, random_state=42)
df_copy['cluster'] = kmeans.fit_predict(df_copy.drop(columns='PRICE'))

cluster_counts = df_copy['cluster'].value_counts()
print(cluster_counts)


cluster
3    1885
6    1534
1    1406
0     915
5     278
4      29
2       1
Name: count, dtype: int64


In [160]:
# 클러스터 2에 해당하는 데이터 삭제
df_copy = df_copy[df_copy['cluster'] != 2]

In [168]:
# K-means 클러스터링 다시 수행
kmeans = KMeans(n_clusters=10, random_state=42)
df_copy['cluster'] = kmeans.fit_predict(df_copy.drop(columns=['PRICE', 'cluster']))

# 각 클러스터별 데이터 개수 출력
print(df_copy['cluster'].value_counts())

cluster
6    1014
9    1014
1     938
2     869
8     689
4     669
0     516
7     236
3      82
5      20
Name: count, dtype: int64


In [169]:
# 2. 클러스터별 데이터 분리 및 모델 학습
models = {}
for cluster_label in df_copy['cluster'].unique():
    # 클러스터별 데이터 추출
    cluster_data = df_copy[df_copy['cluster'] == cluster_label]
    
    # 특성과 타겟 분리
    X = cluster_data.drop(columns=['PRICE', 'cluster'])
    y = cluster_data['PRICE']
    
    # 훈련 및 테스트 데이터 분할
    # 훈련 및 테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=None)
    
    # XGBoost 모델 초기화 및 학습
    xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_reg.fit(X_train, y_train)
    
    # 모델 저장
    models[cluster_label] = xgb_reg

# 학습된 모델 반환
print(models)


{6: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...), 1: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric

In [170]:
from sklearn.metrics import mean_squared_error

# 각 클러스터에 대한 모델의 MSE 저장
mse_scores = {}
for cluster_label, model in models.items():
    # 클러스터별 데이터 추출
    cluster_data = df_copy[df_copy['cluster'] == cluster_label]
    
    # 특성과 타겟 분리
    X = cluster_data.drop(columns=['PRICE', 'cluster'])
    y = cluster_data['PRICE']
    
    # 모델 예측
    y_pred = model.predict(X)
    
    # MSE 계산
    mse = mean_squared_error(y, y_pred)
    mse_scores[cluster_label] = mse

# 각 클러스터에 대한 모델의 MSE 출력
print("각 클러스터별 모델의 MSE:")
for cluster_label, mse in mse_scores.items():
    print(f"Cluster {cluster_label}: {mse**0.5}")


각 클러스터별 모델의 MSE:
Cluster 6: 254.5559373936107
Cluster 1: 371.4877023027201
Cluster 8: 157.36925394449796
Cluster 0: 2749.740647319286
Cluster 2: 153.81167953127823
Cluster 9: 191.00919245695079
Cluster 4: 349.13641793364076
Cluster 7: 114.04600941896341
Cluster 3: 370.3528254153387
Cluster 5: 824.090894368945


In [172]:
df_copy[df_copy['cluster']==7]['PRICE'].mean()

537.9385593220339